In [ ]:
from transformers import CLIPProcessor, TFCLIPModel
from PIL import Image
import tensorflow as tf

In [ ]:
model = TFCLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


In [ ]:
import os
import numpy as np
folder_path = "COCO_dataset/images"
image_paths = [os.path.join(folder_path, img_name) for img_name in os.listdir(folder_path) if img_name.endswith((".jpg", ".jpeg", ".png"))]
image_embeddings_dict = {}


In [ ]:
batch_size = 64
for i in range(0, len(image_paths), batch_size):
    batch_image_paths = image_paths[i:i+batch_size]


    images = [Image.open(img_path) for img_path in batch_image_paths]
    inputs = processor(images=images, return_tensors="tf", padding=True)


    image_embeddings = model.get_image_features(**inputs)


    image_embeddings = tf.math.l2_normalize(image_embeddings, axis=1)


    for img_path, embedding in zip(batch_image_paths, image_embeddings):
        img_name = os.path.basename(img_path)  
        image_embeddings_dict[img_name] = embedding.numpy()

In [ ]:
image_embeddings_dict['COCO_train2014_000000563559.jpg']

In [10]:
data_array = np.array(list(image_embeddings_dict.items()), dtype=object)
np.save('image_to_embedding.npy', data_array)